# PRÁCTICA RAG

Las herramientas que el agente puede ocupar incluyen servicios de búsqueda en internet que permiten extender el conocimiento no disponible por la LLM ya que añadimos esos hechos al contexto de la consulta. Pero veremos que estos esquemas pueden extenderse también con mecanismos de búsqueda como lo son las bases de datos vectoriales.

Gracias a estos medio podemos realizar un amplia recolección de información previo a su envío a la LLM para obtener el resultado final.

In [45]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=r"C:\Users\ST09\Documents\GitHub\STEMIA\Módulo 5 - Retrieval Augmented Generation (RAG)\Fundamentos de los Sistemas RAG\Práctica\.env", override=True)

True

In [46]:
import os

# Verificar si la API key está configurada
api_key = os.getenv("API_KEY_OPENAI")
if api_key:
    print(f"✓ API_KEY_OPENAI configurada (longitud: {len(api_key)} caracteres)")
else:
    print("✗ API_KEY_OPENAI no encontrada")
    print("Asegúrate de que el archivo .env contiene: API_KEY_OPENAI=tu_clave")

✓ API_KEY_OPENAI configurada (longitud: 164 caracteres)


No es tarea trivial ya que encontrar información pertinente para resolver nuestra consulta requiere de encontrar la información relevante de nuestra base de conocimiento. Esto puede realizarse de distintas formas, mediante palabras clave o metadatos durante la consulta aunque las bases de datos vectoriales han sido uno de los puntos clave en la evolución de estos sistemas.

Empleamos bases de datos vectoriales ya que las LLM nos ofrecen la opción de trabajar con los **embeddings** que sabemos aportan un vector denso de características semánticamente relevante. Por lo tanto, podemos realizar una búsqueda por vectores proximos entendiendo que los textos que representan esos vectores serán clave para resolver la consulta en cuestión.

#### Vector search

Sin desplegar grandes soluciones, [FAISS](https://faiss.ai/) nos ofrece un medio sencillo por el que probar esta interacción. Elegiremos primeramente una fuente de la que leer la información.

In [47]:
from langchain_community.document_loaders import WebBaseLoader  ## ojo librería langchain

loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
documents = loader.load()

También deberemos elegir un modelo capaz de construir los embeddings dado un texto. Este mismo modelo será consultado ante una pregunta para obtener los textos o mejor dicho, vectores, que ofrecen la información relevante para nuestra consulta.

In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Creamos embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Creamos vectorstore FAISS
vectorstore = FAISS.from_documents(documents, embedding_model)

print("Vectorstore creado con éxito.")


Vectorstore creado con éxito.


Una vez estas piezas están dispuestas solo tenemos que invocar a la cadena de acciones necesarias. Para simplificar esta operativa LangChain ya dispone de este mecanismo implementado de forma que es sencillo construir el grafo correspondiente.

In [ ]:
# !pip uninstall -y langchain langchain-core langchain-community langchain-openai
# !pip install -U "langchain==0.3.*" "langchain-core==0.3.*" "langchain-community==0.3.*" "langchain-openai==0.2.*" "pydantic>=2,<3"
#! pip uninstall -y openai
#! pip install -U "openai>=1.40.0"
# ! pip install -U "typing_extensions>=4.12.2"


In [37]:
import os
from dotenv import load_dotenv
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hola!"}]
)

In [49]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [50]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",   
    temperature=0.0,
    timeout=60,
    max_retries=2,
)


Retrieval (recuperación) → buscar la información relevante.

Generation (generación) → usar un modelo LLM para generar la respuesta con esa información.

In [51]:
# Busca la infor en vectorstore y devuelve la más relevante 
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [52]:
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente experto. Responde SOLO con el contexto. "
     "Si no está en el contexto, dilo claramente. Cita [S1], [S2], ..."),
    ("human", "Pregunta: {question}\n\nContexto:\n{context}")
])

In [53]:
def _format_context(docs, max_chars=1200):
    partes = []
    for i, d in enumerate(docs, start=1):
        src = d.metadata.get("source") or d.metadata.get("url") or d.metadata.get("path") or "desconocido"
        txt = (d.page_content or "").replace("\n", " ").strip()
        if len(txt) > max_chars:
            txt = txt[:max_chars] + "..."
        partes.append(f"[S{i}] Fuente: {src}\n{txt}")
    return "\n\n".join(partes)

In [56]:
def consultar_bd(question: str, k: int = 5):
    docs = retriever.invoke(question)
    ctx = _format_context(docs)
    messages = prompt.format_messages(question=question, context=ctx)
    ai_msg = llm.invoke(messages)
    return {"answer": ai_msg.content, "source_documents": docs}

In [57]:
# Ejemplo:

result = consultar_bd("¿Qué es LangChain y para qué sirve?")
print(result["answer"])
for i, d in enumerate(result["source_documents"], start=1):
    print(f"[S{i}] ->", d.metadata.get("source") or d.metadata.get("url") or d.metadata.get("path"))

LangChain es una herramienta que permite construir aplicaciones utilizando modelos de lenguaje (LLM) y se utiliza para crear chatbots, sistemas de preguntas y respuestas, motores de búsqueda semántica, y aplicaciones de generación aumentada por recuperación (RAG), entre otros. [S1]
[S1] -> https://python.langchain.com/docs/introduction/


Podemos ver los extractos de texto rescatados que han sido añadidos a la consulta.

In [58]:
print(result["source_documents"])

[Document(id='488c5d42-0ba6-4cee-85e0-557481c2b2b8', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'introduction | 🦜️🔗 LangChain', 'description': '⚠️ THESE DOCS ARE OUTDATED. Visit the new v1.0 docs', 'language': 'en'}, page_content='\n\n\n\n\nintroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\n\nSkip to main content⚠️ THESE DOCS ARE OUTDATED. Visit the new v1.0 docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseindexBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesindexHow to use tools in a chainHow to use 